Dont know what i think so far. I am adding more parameters, but now the can learn different non-linearities. Adding more parameters will lead to over fitting? But i can just reduce the nets size?

I could tie weights across
* neurons?
* read/write????

Based on 
* Balduzzi - Strongly typed RNNs
* and Neural Turing Machines.


Permutations
* Additive, multiplication
* Walks through weight space, aka the environment

Analogy
* Memory = environment
* Weights = position in environment
* Neurons = ants, particles, ??
* 

Other questions/problems
* Make the number of time steps a variable and add it as a regulariser in loss?
* Unstable... how can you make more stable systems? Negative feedback?

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from fuel.datasets import MNIST
mnist = MNIST(('train',))
state = mnist.open()

In [3]:
# Hyper parameters
n_inputs = 28
n_hidden = 0
n_outputs = 10
m = n_inputs + n_hidden + n_outputs #number of memory cells

n = 10#number of neurons

time_steps = 5

In [4]:
#Neurons that dont have a state
class Neurons():
    def __init__(self,n):
        self.alphas = tf.Variable(tf.random_normal((n,1),dtype=np.float32),'alpha')
        self.betas = tf.Variable(np.ones((n,1),dtype=np.float32),'beta')
        
    def forward(self,phi):
        #normalise to stop exploding/diminishing state values
        m = tf.reduce_mean(phi)
        v = tf.reduce_mean(tf.square(phi-m))
        return  tf.nn.elu(self.alphas + (phi-m)*self.betas/v) #tf.exp(phi*self.betas) *self.alphas 

In [5]:
x = tf.placeholder(tf.float32,shape = [n_inputs,n_inputs])
L = tf.placeholder(tf.int64, shape = [1,1])
y_ = tf.reshape(tf.one_hot(L,10,1.0,0.0),[n_outputs,1])

#The environment where signals interact.
memory = tf.Variable(np.ones((m,1),dtype=np.float32),'Memory') 

#The weights. Do we want these to be symmetric? Do we want to learn these?
read = tf.Variable(tf.random_normal((n,m)),'ReadConnectionWeights')
write = tf.Variable(tf.random_normal((m,n)),'WriteConnectionWeights')
biases = tf.Variable(-np.ones((n,1),dtype=np.float32),'Biases')

neurons = Neurons(n)

In [6]:
def cell(inputs, memory):
    #with tf.VariableScope('Basic'):
    # Inputs
    i = tf.slice(memory, [0,0],[n_inputs,1])
    j = tf.slice(memory, [n_inputs,0], [m-n_inputs,1])
    i = i + inputs
    mem = tf.concat(0,[i,j])

    ### Dynamics
    #the inputs to our neurons. 
    phi = tf.matmul( read , mem ) + biases #nxm x mx1 = nx1
    #the inputs to our memory cells. 
    psi = tf.matmul( write , neurons.forward(phi) ) #mxn x nx1 = mx1

    return psi*memory

In [7]:
#Since I have rolled this operation up with in the tf graph, it is hard to query it for values
with tf.variable_scope("RNN"):
    for t in range(time_steps):
        if time_steps > 0:
            tf.get_variable_scope().reuse_variables()
        memory = cell(tf.slice(x, [0,t],[-1,1]), memory)

In [8]:
### Classification
s = tf.slice(memory, [m-n_outputs,0],[n_outputs,1]) #take the outputs
W_soft = tf.Variable(tf.random_normal((n_outputs,n_outputs)))
b_soft = tf.Variable(np.zeros((n_outputs),dtype = np.float32))

soft = tf.reshape(tf.matmul(W_soft,s),[1,n_outputs]) + b_soft
m = tf.reduce_mean(soft)
v = tf.reduce_mean(tf.square(soft-m))
softin = tf.abs((soft - m)/v)
y = tf.nn.softmax(softin)
#hmm. have to do some nasty hacks to get some decent output values. positive, and small (ish)
#how do conv nets manage this??

In [9]:
### Loss and optimisation
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[0]))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,0), tf.argmax(y_,0))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [11]:
ans = sess.run([memory,y,soft,softin],feed_dict={x : np.ones((n_inputs,n_inputs),dtype = np.float32),L : np.ones((1,1),dtype = np.int64 )})
for a in ans:
    print(a.shape)

(38, 1)
(1, 10)
(1, 10)
(1, 10)


In [12]:
for a in ans:
    print(a)

[[  1.05607948e+01]
 [  1.45409012e-03]
 [ -4.39483490e+02]
 [  1.54031052e+02]
 [ -6.07794714e+00]
 [  6.68625608e-02]
 [  1.51745346e+02]
 [  8.96846313e+01]
 [  5.39112427e+02]
 [ -2.07207661e+01]
 [  1.22115060e-04]
 [ -1.33262050e+00]
 [  8.22290726e+01]
 [  6.56104851e+00]
 [ -4.54012756e+01]
 [ -8.60629517e+02]
 [  7.24232788e+01]
 [  1.26268864e+01]
 [ -3.63664702e-02]
 [  2.59771149e+02]
 [  2.33434830e+02]
 [ -1.52324271e+00]
 [ -2.18608008e+03]
 [  8.38435974e+01]
 [  3.88211831e-02]
 [  2.36482956e+02]
 [ -3.42604704e-02]
 [ -5.08655071e-01]
 [ -1.04268627e+01]
 [  8.91410828e+00]
 [  1.45696020e+01]
 [ -2.12244614e+02]
 [  3.69294930e+00]
 [  6.27843689e+02]
 [  2.41797066e+01]
 [  3.51288989e+03]
 [  3.05528736e+01]
 [  3.71107758e+02]]
[[ 0.10000157  0.0999779   0.10000884  0.10000444  0.09998711  0.09998541
   0.10002433  0.10001744  0.10001078  0.09998216]]
[[-1605.20715332   874.67248535  5127.13867188  4665.79492188
   2850.62866211    87.9720459  -3987.88769531  602

In [13]:
batch_size = 1
from fuel.schemes import ShuffledScheme
scheme = ShuffledScheme(examples=int(mnist.num_examples/100), batch_size=batch_size)
for i,request in enumerate(scheme.get_request_iterator()):
    data,labels = mnist.get_data(state=state, request=request)
    feed = feed_dict= {x:data.reshape(28,28),L:labels}
    sess.run(train_step, feed)
    if i%10 == 0:
        acc,cross=sess.run([accuracy,cross_entropy], feed)
        print(i,': Accuracy = ',acc,': Cross entropy = ',cross)

0 : Accuracy =  0.0 : Cross entropy =  2.30259
10 : Accuracy =  0.0 : Cross entropy =  2.30259
20 : Accuracy =  0.0 : Cross entropy =  2.30259
30 : Accuracy =  0.0 : Cross entropy =  2.30259
40 : Accuracy =  0.0 : Cross entropy =  2.30259
50 : Accuracy =  0.0 : Cross entropy =  2.30259
60 : Accuracy =  0.0 : Cross entropy =  2.30259
70 : Accuracy =  0.0 : Cross entropy =  2.30259
80 : Accuracy =  0.0 : Cross entropy =  2.30259
90 : Accuracy =  0.0 : Cross entropy =  2.30259
100 : Accuracy =  0.0 : Cross entropy =  2.30259
110 : Accuracy =  1.0 : Cross entropy =  2.30259
120 : Accuracy =  0.0 : Cross entropy =  2.30259
130 : Accuracy =  0.0 : Cross entropy =  2.30259
140 : Accuracy =  0.0 : Cross entropy =  2.30259
150 : Accuracy =  0.0 : Cross entropy =  2.30259
160 : Accuracy =  0.0 : Cross entropy =  2.30259
170 : Accuracy =  1.0 : Cross entropy =  2.30259
180 : Accuracy =  0.0 : Cross entropy =  2.30259
190 : Accuracy =  0.0 : Cross entropy =  2.30259
200 : Accuracy =  0.0 : Cross e